# Alternate implementation of MEMO graph (dict or matrix)

In [1]:
import numpy as np
import pandas as pd
import networkx as nx
from networkx.algorithms import bipartite
import matplotlib.pyplot as plt
from itertools import combinations
import itertools
import copy
from tqdm.notebook import tqdm,trange
import multiprocessing
import concurrent.futures
import time
import random
import os
import gc
random.seed(1234)

In [2]:
pwd

'/media/ml-lab/Rafsan Elements 1TB/pancancer_stages_epistasis/version yii - MEMO/memo/src'

In [3]:
## Functions

def load_cosmic_genes(cosmic_infile):
    """Load cosmic gene list"""
    with open(cosmic_infile, 'r') as f:
        return [line.split()[0] for line in f.readlines()[1:]]

def load_intact_edges(intact_edge_file,intact_index_file):
    """Load intact edges/gene pairs"""
    with open(intact_index_file, 'r') as f:
        indices = {line.split()[0]:line.split()[1] for line in f.readlines()}
    
    with open(intact_edge_file, 'r') as f:
        edges = [(indices[line.split()[0]].upper(),indices[line.split()[1]].upper()) for line in f.readlines()]
    print(len(edges))
    
    intact_genes_list = list(sorted(indices.values()))
    print(intact_genes_list[:5])
    
    return edges,indices

def random_bip(M,adj_matrix,edges,Q=100):
    """ swap edges Q*len(edges times) while preserving the number of edges"""
    steps = Q*len(edges)
    pairs = random.choices(tuple(itertools.combinations(adj_matrix,2)),k=steps)
#     print(pairs)
#     raise Exception()
    for g_idx1,g_idx2 in pairs:
        
        ## random patients from adj matrix
        p_idx1 = random.choice(adj_matrix[g_idx1])
        p_idx2 = random.choice(adj_matrix[g_idx2])
        
        if M[p_idx1][g_idx2]==0 and M[p_idx2][g_idx1]==0:
#             print(g_idx1,p_idx1,g_idx2,p_idx2,M[p_idx1][g_idx1],M[p_idx2][g_idx2])
            
            M[p_idx1][g_idx1] = 0
            M[p_idx2][g_idx2] = 0
            adj_matrix[g_idx1].remove(p_idx1)
            adj_matrix[g_idx2].remove(p_idx2)

            M[p_idx1][g_idx2] = 1
            M[p_idx2][g_idx1] = 1
            adj_matrix[g_idx1].append(p_idx2)
            adj_matrix[g_idx2].append(p_idx1)
    
    del pairs
    gc.collect()
            
    return M,adj_matrix

def calculate_mscore(gene_comb, d_score):
    """calculate len of union pf patients where genes mutate"""
    d_gp = {}
    for g1,g2 in gene_comb:
        d_gp[(g1,g2)] = len(set().union(d_score[g1],d_score[g2]))
        
#     print(d_gp[(2,5)],d_gp[(0,1)],d_gp[(1,2)])
    return(d_gp)

def calculate_mscore_parallel(gene_comb,M, adj_dict,edges):
    """calculate len of union pf patients where genes mutate in parallel"""
    H = copy.deepcopy(M)
    d = copy.deepcopy(adj_dict)
    H_new,d_new = random_bip(H,d,edges)
    d_gp = {}
    for g1,g2 in gene_comb:
        d_gp[(g1,g2)] = len(set().union(d_new[g1],d_new[g2]))
    
    del H,d,H_new,d_new
    gc.collect()
        
#     print("tin mscore",d_gp[(12,15)],d_gp[(102,150)],d_gp[(12,20)],d_gp[(0,5)])
    return(d_gp)

def get_pvalue(G,iters=10000):
    "main function to get pvalues of all combinations of genes"
    row,col = G.shape
    gene_comb = tuple(itertools.combinations(range(col),2))
    mscore_dict = {k:0 for k in gene_comb}
#     print(gene_comb[:10])
    edges = np.transpose(G.nonzero())
    dict_gvp = {g_idx:[] for g_idx in range(col)}
    for p,g in edges:
        dict_gvp[g].append(p)
        
    mscore_orig = calculate_mscore(gene_comb, dict_gvp)

    for i in trange(iters):
        H = copy.deepcopy(G)
        d = copy.deepcopy(dict_gvp)
        H_new,d_new = random_bip(H,d)
#         print(H_new, np.transpose(H_new.nonzero()))
#         print(H)
        
        mscore_temp = calculate_mscore(gene_comb,d_new)
        
        for k,v in mscore_temp.items():
            if mscore_orig[k]<=v:
                mscore_dict[k]+=1
                
    
    return {k:float(v)/float(iters) for k,v in mscore_dict.items()}

def get_pvalue_parallel(G,iters=10000,chunks=4):
    "main function to get pvalues of all combinations of genes in parallel"
    row,col = G.shape
    gene_comb = tuple(itertools.combinations(range(col),2))
#     print(gene_comb)
#     raise Exception("ERR")
    
    mscore_dict = {k:0 for k in gene_comb}
#     print(gene_comb[:10])
    edges = np.transpose(G.nonzero())
    dict_gvp = {g_idx:[] for g_idx in range(col)}
    for p,g in edges:
        dict_gvp[g].append(p)
        
    mscore_orig = calculate_mscore(gene_comb, dict_gvp)
#     print('ORIG',mscore_orig[(12,15)],mscore_orig[(102,150)],mscore_orig[(12,20)],mscore_orig[(0,5)])
    with concurrent.futures.ProcessPoolExecutor() as EXC:
        assert iters%chunks ==0
        iters_in_chunk = int(iters/chunks)
        for i in trange(chunks):
            
            res = [EXC.submit(calculate_mscore_parallel,gene_comb,G,dict_gvp,edges) for i in range(iters_in_chunk)]

    #         print(H_new, np.transpose(H_new.nonzero()))
    #         print(H)

            for f in concurrent.futures.as_completed(res):
                mscore_temp = f.result()
    #             print(mscore_temp)
                for k,v in mscore_temp.items():
                    if mscore_orig[k]<=v:
                        mscore_dict[k]+=1
            
            del res,mscore_temp
            gc.collect()
    
    del gene_comb
    gc.collect()
    
    return {k:float(v)/float(iters) for k,v in mscore_dict.items()}

def get_pvalue_parallel_minus_ncgncg(G,ref_genes, nonref_genes,iters=10000,chunks=4):
    "main function to get pvalues of all combinations of genes in parallel, minus non cosmic gene pairs"
    row,col = G.shape
    gene_comb = tuple((g1,g2) for g1,g2 in tuple(itertools.combinations(range(col),2)) if g1 in ref_genes or g2 in ref_genes)
#     print('gene_comb',gene_comb)
#     raise Exception("ERR")
    
    mscore_dict = {k:0 for k in gene_comb}
#     print(gene_comb[:10])
    edges = np.transpose(G.nonzero())
    dict_gvp = {g_idx:[] for g_idx in range(col)}
    for p,g in edges:
        dict_gvp[g].append(p)
        
    mscore_orig = calculate_mscore(gene_comb, dict_gvp)
#     print('ORIG',mscore_orig[(12,15)],mscore_orig[(102,150)],mscore_orig[(12,20)],mscore_orig[(0,5)])
    with concurrent.futures.ProcessPoolExecutor() as EXC:
        assert iters%chunks ==0
        iters_in_chunk = int(iters/chunks)
        for i in trange(chunks):
            
            res = [EXC.submit(calculate_mscore_parallel,gene_comb,G,dict_gvp,edges) for i in range(iters_in_chunk)]

    #         print(H_new, np.transpose(H_new.nonzero()))
    #         print(H)

            for f in concurrent.futures.as_completed(res):
                mscore_temp = f.result()
    #             print(mscore_temp)
                for k,v in mscore_temp.items():
                    if mscore_orig[k]<=v:
                        mscore_dict[k]+=1
            
            del res,mscore_temp
            gc.collect()
    
    del gene_comb
    gc.collect()
    
    return {k:float(v)/float(iters) for k,v in mscore_dict.items()}

In [4]:
# cosmic_infile = '../data/Census_allFri_Apr_26_12_49_57_2019.tsv'
# cosmic_genes = load_cosmic_genes(cosmic_infile)
# ref_genes = set([i for i in range(len(genes)) if genes[i] in cosmic_genes])
# nonref_genes = set([i for i in range(len(genes)) if genes[i] not in cosmic_genes])


In [ ]:
intact_edge_file = '../data/intact_nodupl_edge_file.txt'
intact_index_file = '../data/intact_nodupl_index_file.txt'
cosmic_infile = '../data/Census_allFri_Apr_26_12_49_57_2019.tsv'

outpath = '../out/memo_mutation_filtered_ep_data/'
inpath = '../data/binary_matrices_all_genes_ep_mutation_filtered/'
if not os.path.exists(outpath):
    os.makedirs(outpath)

    
# load intact
intact_edges,intact_indices = load_intact_edges(intact_edge_file,intact_index_file)
cosmic_genes = load_cosmic_genes(cosmic_infile)

#dividing_factor= 10000.0
##['BLCA', 'BRCA', 'COADREAD', 'LUAD', 'LUSC', 'SKCM', 'STAD', 'UCEC']
cancer_types = ['SKCM']#['LUAD', 'LUSC', 'SKCM', 'STAD', 'UCEC']
threshold_values = [20]#,10,5]
cols = ['gene1','gene2', 'pvalue']

for t in tqdm(threshold_values):

    for c in cancer_types:
        
        print(c,t)
        infile = inpath +'{}_TML_binary_sm.txt'.format(c)
        outfile = outpath + '{}_memo_result_mutations_all_genes_{}.txt'.format(c,t)
        outfile_intact = outpath + '{}_memo_pairs_intact_filtered_subset{}.txt '.format(c,t)
    

        # Dataframe cration and mutation filtering
        df = pd.read_csv(infile,sep='\t',header=0,index_col=0)
        df.drop('y',1,inplace=True)
#         print(df.shape)
#         print(df.head())

        df.drop([col for col, val in df.sum().iteritems() if val <= t], axis=1, inplace=True)
#         print(df.shape)
#         df.head()

        ##Gene and patients
        patients = df.index.tolist()
        genes = df.columns.tolist()
        ref_genes = set([i for i in range(len(genes)) if genes[i] in cosmic_genes])
        nonref_genes = set([i for i in range(len(genes)) if genes[i] not in cosmic_genes])


        ## Assign indices to gene combinations
        d_comb={}
        count=0
        comblist=list(combinations(genes,2))

        for g1, g2 in comblist:
            d_comb[count]=(g1,g2)
            count+=1

        ## get matrix from df
        G_matrix = df.to_numpy()
        
        #for ncgncg
#         dict_res = get_pvalue_parallel_minus_ncgncg(G_matrix,ref_genes,nonref_genes,chunks=500)

        dict_res = get_pvalue_parallel(G_matrix,chunks=50)

        ## Create dataframe from dict
        l_out = []
        for k,v in dict_res.items():
            g1=genes[k[0]]
            g2=genes[k[1]]
            l_out.append([g1,g2,v])

        df_res = pd.DataFrame(l_out,columns=cols)
        print("df res",df_res.head(), df_res.shape)
        
        # Create dictionary for lookup
        g1_list = df_res['gene1'].tolist()
        g2_list = df_res['gene2'].tolist()
        pval = df_res['pvalue'].tolist()
        dict_temp_intact = {g1+'\t'+g2:p for g1,g2,p in zip(g1_list,g2_list, pval)}
        
        list_temp_intact = []
        #edges should have no duplicates
        for e in tqdm(intact_edges, desc='PPI filter'):
            g1 = e[0]
            g2 = e[1]

            if '{}\t{}'.format(g1,g2) in dict_temp_intact:
                pval = dict_temp_intact['{}\t{}'.format(g1,g2)]
                list_temp_intact.append([g1,g2,pval])

            elif '{}\t{}'.format(g2,g1) in dict_temp_intact:
                pval = dict_temp_intact['{}\t{}'.format(g2,g1)]
                list_temp_intact.append([g1,g2,pval])

#         print(len(list_temp_intact))
        df_res_intact = pd.DataFrame(list_temp_intact,columns=cols)
        print("df res intact",df_res_intact.head(), df_res_intact.shape)
        
        #output file
        df_res.to_csv(outfile, sep='\t',columns=cols)
        df_res_intact.to_csv(outfile_intact, sep='\t', columns=cols)
        

        

103520
['""CHEBI', '100147744', '1B', '1EFV', '1KLA']


SKCM 20


In [ ]:
genes[21],cosmic_genes